In [29]:
import pandas as pd
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import bs4.element
import warnings
warnings.filterwarnings('ignore')
import re
import time
from datetime import datetime
from tqdm import tqdm

In [30]:
def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', str(d))
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\']'           
    text = re.sub(pattern, '', text)
    pattern = '[\"]'            
    text = re.sub(pattern, '', text)
    return text  

In [31]:
f = ['경제', '국제시장', '기술혁신', '로켓', '무기수출', '미국', '미사일', '비행체부품', '사회', '연구개발', '우주기술', '전쟁', '전투기', '정치', '코스피200', '한화에어로스페이스', '항공엔진', '항공우주 정책과 규제', '해양']

In [136]:
folder = '코스피200'

In [137]:
folders = os.listdir('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}'.format(folder))

df_all = pd.DataFrame()
for i in range(0,len(folders)):
    if folders[i].split('.')[1] == 'csv':
        file = 'D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\'.format(folder)+folders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

df_all

,date,title,linkcontent,link,content
0,2021-06-28 16:09:11,"편의점 3사, 식약처와 '균형 식단 알리기 캠페인' 맞손",NaN,https://n.news.naver.com/mnews/article/629/000...,[\n\n\n\n\n편의점 3사가 식품의약품안전처와 손잡고 고객들의 균형잡힌 식사를...
1,2021-06-30 11:14:51,"GS 편의점·슈퍼·홈쇼핑 합쳤다…""당일 배송으로 승부할 것""",NaN,https://n.news.naver.com/mnews/article/025/000...,[\n \n\n\n\nGS리테일과 GS홈쇼핑이 합친 통합 GS리테일호가 7월1일 ...
2,2021-06-29 11:16:07,할인에 할인을 더하니 비빔면 190원…마트보다 치열한 편의점 가격 전쟁,NaN,https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n이마트24 직원이 7월 7일까지 1+4 특가 판매를 하는 비말차...
3,2021-06-29 19:50:00,"TV도, 폰도 무인판매…""눈치 안보고 구경"" 1만명 다녀갔다",NaN,https://n.news.naver.com/mnews/article/008/000...,[\n\t\t\t[편집자주] 무인매장 시대가 성큼 다가왔다. 코로나19 사태 이후 ...
4,2021-06-30 13:09:03,"고양지역자활센터, CU편의점 2호점 오픈식 진행",NaN,https://n.news.naver.com/mnews/article/417/000...,"[\n\n\n\n\n고양지역자활센터, CU편의점 2호점 오픈식. / 사진제공=고양시..."
...,...,...,...,...,...
0,2021-06-30 11:40:00,내수회복 기대감…활짝 웃는 중소형주 펀드,NaN,https://n.news.naver.com/mnews/article/277/000...,[\n경기민감 업종 대거 포진연초 이후 수익률 15.32%국내·해외 주식형 수익률 ...
0,2021-06-30 18:24:01,콘택트 시대 부활…백화점·레저 주목…포스트 코로나 新성장주 ‘메타버스’,NaN,https://n.news.naver.com/mnews/article/024/000...,[\n\t\t\t증권가는 ‘돈 냄새’를 누구보다 빠르게 맡는 곳이다. 코로나19 백...
1,2021-06-28 16:07:57,"코스피 0.03% 하락 마감…이틀째 3,300선 유지(종합)",NaN,https://n.news.naver.com/mnews/article/001/001...,"[\n\n\n\n\n코스피 3,300선 유지(서울=연합뉴스) 임헌정 기자 = 코스피..."
0,2023-07-05 09:16:01,중국서만 나이키급… 휠라의 고민 [컴퍼니+],NaN,https://n.news.naver.com/mnews/article/665/000...,[\n더스쿠프 컴퍼니 인사이트 휠라 엇갈린 지역별 실적 중국서 프리미엄 브랜드로 ...


In [138]:
df_all.drop('link', axis=1, inplace=True)
# df_all.drop('Unnamed: 0', axis=1, inplace=True)
df_all.drop('linkcontent', axis=1, inplace=True)

In [139]:
df_all.dropna(axis=0, inplace=True)

In [140]:
df_all['content'] = df_all['content'].apply(text_clean)
df_all['content'] = df_all['content'].apply(clean_text)
df_all['title'] = df_all['title'].apply(text_clean)
df_all['title'] = df_all['title'].apply(clean_text)

df_all

,date,title,content
0,2021-06-28 16:09:11,편의점 사 식약처와 균형 식단 알리기 캠페인 맞손,는 식약처 블로그나 공식 페이스북 및 인스타그램에서 확인할 수 있다는 한 끼 식단...
1,2021-06-30 11:14:51,편의점슈퍼홈쇼핑 합쳤다당일 배송으로 승부할 것,리테일과 홈쇼핑이 합친 통합 리테일호가 월일 출범한다 리테일 편의점 와 슈퍼마켓 ...
2,2021-06-29 11:16:07,할인에 할인을 더하니 비빔면 원마트보다 치열한 편의점 가격 전쟁,이마트 직원이 월 일까지 특가 판매를 하는 비말차단마스크를 진열하고 있다 이마트 제...
3,2021-06-29 19:50:00,도 폰도 무인판매눈치 안보고 구경 만명 다녀갔다,지난 월 서울 종로구에 문을 연 유플러스의 호 무인매장도 이런 고민의 결과다 박성...
4,2021-06-30 13:09:03,고양지역자활센터 편의점 호점 오픈식 진행,고양지역자활센터 편의점 호점 오픈식 제공고양시고양시시장 이재준는 지난 일 저소득층의...
...,...,...,...
0,2021-06-30 11:40:00,내수회복 기대감활짝 웃는 중소형주 펀드,경기민감 업종 대거 포진연초 이후 수익률 국내해외 주식형 수익률 추월경제 활동 회복...
0,2021-06-30 18:24:01,콘택트 시대 부활백화점레저 주목포스트 코로나 성장주 메타버스,오프라인 활동콘택트 부활 골프캠핑 등 레저 활동백화점 북적 백신 접종으로 인한 안...
1,2021-06-28 16:07:57,코스피 하락 마감이틀째 선 유지종합,김아람 기자 코스피가 일 약보합으로 마감하며 이틀째 선을 유지했다 이날 코스피는 ...
0,2023-07-05 09:16:01,중국서만 나이키급 휠라의 고민 컴퍼니,더스쿠프 컴퍼니 인사이트 휠라 엇갈린 지역별 실적 중국서 프리미엄 브랜드로 안착중국...


In [141]:
df_all.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\0. merge\\{}_merge.csv'.format(folder), index=False)